<a href="https://colab.research.google.com/github/OlhaZahrebelna/python_for_ds_task/blob/main/Zahrebelna_Olha_%22HW_2_2_%D0%9D%D0%B5%D0%B7%D0%B1%D0%B0%D0%BB%D0%B0%D0%BD%D1%81%D0%BE%D0%B2%D0%B0%D0%BD%D0%B0_%D0%B1%D0%B0%D0%B3%D0%B0%D1%82%D0%BE%D0%BA%D0%BB%D0%B0%D1%81%D0%BE%D0%B2%D0%B0_%D0%BA%D0%BB%D0%B0%D1%81%D0%B8%D1%84%D1%96%D0%BA%D0%B0%D1%86%D1%96%D1%8F_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [1]:
import pandas as pd
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import balanced_accuracy_score, classification_report
from sklearn.svm import SVC

from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTENC
from imblearn.combine import SMOTETomek

In [2]:
df = pd.read_csv('customer_segmentation_train.csv')
df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [4]:
df.isna().sum()

,0
ID,0
Gender,0
Ever_Married,140
Age,0
Graduated,78
Profession,124
Work_Experience,829
Spending_Score,0
Family_Size,335
Var_1,76


In [5]:
for i in df.select_dtypes(include='object').columns:
    print(i)
    print(df[i].unique())

Gender
['Male' 'Female']
Ever_Married
['No' 'Yes' nan]
Graduated
['No' 'Yes' nan]
Profession
['Healthcare' 'Engineer' 'Lawyer' 'Entertainment' 'Artist' 'Executive'
 'Doctor' 'Homemaker' 'Marketing' nan]
Spending_Score
['Low' 'Average' 'High']
Var_1
['Cat_4' 'Cat_6' 'Cat_7' 'Cat_3' 'Cat_1' 'Cat_2' nan 'Cat_5']
Segmentation
['D' 'A' 'B' 'C']


In [6]:
y = df['Segmentation']
X = df.drop(columns=['Segmentation', 'ID'])

le = LabelEncoder()
y_enc = le.fit_transform(y)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y_enc, test_size=0.2, random_state=42, stratify=y_enc
)

print("Train size:", X_train.shape, " Test size:", X_test.shape)
print("Train class distribution:", Counter(y_train))

Train size: (6454, 9)  Test size: (1614, 9)
Train class distribution: Counter({np.int64(3): 1814, np.int64(0): 1578, np.int64(2): 1576, np.int64(1): 1486})


**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

Препроцесінг

Для **SMOTENC** категоріальні фічі мають бути **однією колонкою з цілочисельними кодами**, тому:

числові: SimpleImputer(median) + StandardScaler

категоріальні: SimpleImputer(most_frequent) + OrdinalEncoder

**Spending_Score** має природний порядок: Low < Average < High.

In [8]:
binary_cols  = ["Gender", "Ever_Married", "Graduated"]
ordinal_cols = ["Spending_Score"]
nominal_cols = ["Profession", "Var_1"]

cat_cols = binary_cols + ordinal_cols + nominal_cols
num_cols = [c for c in X_train.columns if c not in cat_cols]

pre_smote = ColumnTransformer(
    transformers=[
        ("num", Pipeline([
            ("imp", SimpleImputer(strategy="median")),
            ("sc", StandardScaler())
        ]), num_cols),

        ("cat", Pipeline([
            ("imp", SimpleImputer(strategy="most_frequent")),
            ("enc", OrdinalEncoder(
                # якщо зʼявиться нова категорія на тесті — ставимо -1
                handle_unknown="use_encoded_value",
                unknown_value=-1
            ))
        ]), cat_cols),
    ],
    remainder="drop"
)

cat_feature_indices = list(range(len(num_cols), len(num_cols) + len(cat_cols)))
cat_feature_indices

[3, 4, 5, 6, 7, 8]

- **SMOTENC** (upsampling)
- **SMOTE-Tomek** (SMOTENC + очищення Tomek links)

In [9]:
X_train_p = pre_smote.fit_transform(X_train)

print("Before:", Counter(y_train))

# SMOTENC
sm = SMOTENC(categorical_features=cat_feature_indices, random_state=42)
X_sm, y_sm = sm.fit_resample(X_train_p, y_train)
print("After SMOTENC:", Counter(y_sm))

# SMOTE-Tomek (з SMOTENC всередині)
st = SMOTETomek(
    smote=SMOTENC(categorical_features=cat_feature_indices, random_state=42),
    random_state=42
)
X_st, y_st = st.fit_resample(X_train_p, y_train)
print("After SMOTE-Tomek:", Counter(y_st))

Before: Counter({np.int64(3): 1814, np.int64(0): 1578, np.int64(2): 1576, np.int64(1): 1486})
After SMOTENC: Counter({np.int64(0): 1814, np.int64(1): 1814, np.int64(2): 1814, np.int64(3): 1814})
After SMOTE-Tomek: Counter({np.int64(2): 1484, np.int64(3): 1484, np.int64(1): 1421, np.int64(0): 1415})


**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [10]:
X_test_p = pre_smote.transform(X_test)

base_lr = LogisticRegression(max_iter=5000, solver="lbfgs")

# ORIGINAL (без ресемплінгу)
ovr_orig = OneVsRestClassifier(base_lr)
ovr_orig.fit(X_train_p, y_train)
pred_orig = ovr_orig.predict(X_test_p)

# SMOTENC
ovr_smote = OneVsRestClassifier(base_lr)
ovr_smote.fit(X_sm, y_sm)
pred_smote = ovr_smote.predict(X_test_p)

# SMOTE-Tomek
ovr_smotetomek = OneVsRestClassifier(base_lr)
ovr_smotetomek.fit(X_st, y_st)
pred_st = ovr_smotetomek.predict(X_test_p)

print("=== OvR LR: ORIGINAL ===")
print("Balanced accuracy:", balanced_accuracy_score(y_test, pred_orig))
print(classification_report(y_test, pred_orig, target_names=le.classes_))

print("=== OvR LR: SMOTENC ===")
print("Balanced accuracy:", balanced_accuracy_score(y_test, pred_smote))
print(classification_report(y_test, pred_smote, target_names=le.classes_))

print("=== OvR LR: SMOTE-Tomek ===")
print("Balanced accuracy:", balanced_accuracy_score(y_test, pred_st))
print(classification_report(y_test, pred_st, target_names=le.classes_))

=== OvR LR: ORIGINAL ===
Balanced accuracy: 0.4761347712009064
              precision    recall  f1-score   support

           A       0.39      0.39      0.39       394
           B       0.41      0.08      0.14       372
           C       0.47      0.64      0.54       394
           D       0.59      0.79      0.67       454

    accuracy                           0.49      1614
   macro avg       0.46      0.48      0.44      1614
weighted avg       0.47      0.49      0.45      1614

=== OvR LR: SMOTENC ===
Balanced accuracy: 0.48321165355852935
              precision    recall  f1-score   support

           A       0.41      0.38      0.40       394
           B       0.36      0.15      0.21       372
           C       0.47      0.63      0.54       394
           D       0.62      0.78      0.69       454

    accuracy                           0.50      1614
   macro avg       0.46      0.48      0.46      1614
weighted avg       0.47      0.50      0.47      1614

=== 

# Не лінійна модель

In [11]:
svc_params = dict(kernel="rbf", C=3, gamma="scale")

In [12]:
clf_smote = SVC(**svc_params)
clf_smote.fit(X_sm, y_sm)
pred_smote = clf_smote.predict(X_test_p)

clf_smotetomek = SVC(**svc_params)
clf_smotetomek.fit(X_st, y_st)
pred_smotetomek = clf_smotetomek.predict(X_test_p)

print("\n=== SVC trained on SMOTENC-resampled train ===")
print("Balanced accuracy:", balanced_accuracy_score(y_test, pred_smote))
print(classification_report(y_test, pred_smote, target_names=le.classes_))

print("\n=== SVC trained on SMOTE-Tomek-resampled train ===")
print("Balanced accuracy:", balanced_accuracy_score(y_test, pred_smotetomek))
print(classification_report(y_test, pred_smotetomek, target_names=le.classes_))


=== SVC trained on SMOTENC-resampled train ===
Balanced accuracy: 0.5250029635475238
              precision    recall  f1-score   support

           A       0.44      0.45      0.44       394
           B       0.43      0.40      0.42       372
           C       0.55      0.53      0.54       394
           D       0.66      0.72      0.69       454

    accuracy                           0.53      1614
   macro avg       0.52      0.53      0.52      1614
weighted avg       0.53      0.53      0.53      1614


=== SVC trained on SMOTE-Tomek-resampled train ===
Balanced accuracy: 0.5284194288973415
              precision    recall  f1-score   support

           A       0.45      0.47      0.46       394
           B       0.43      0.42      0.43       372
           C       0.58      0.51      0.54       394
           D       0.66      0.71      0.68       454

    accuracy                           0.54      1614
   macro avg       0.53      0.53      0.53      1614
weighted 

# Висновки


**Основні метрики**: Macro-F1 і Balanced Accuracy

**Найкраша модель** RBF(нелінійна модель)-SVC + Tomek

Порівняння показало суттєву різницю між лінійною та нелінійною моделями. Для логістичної регресії (OvR) застосування балансування даних через SMOTENC та SMOTE-Tomek дало лише незначні зміни: метрики macro F1 та balanced accuracy майже не зросли. Це означає, що головне обмеження в цьому випадку — лінійність моделі: навіть після ресемплінгу вона не здатна суттєво покращити розділення класів, якщо межі між ними є складнішими.
Натомість після заміни моделі на нелінійну SVC з RBF-ядром якість помітно зросла (зокрема, покращилися показники для “проблемного” класу B), що свідчить про наявність нелінійних залежностей у даних. Найкращий результат серед протестованих підходів дала комбінація SMOTE-Tomek + SVC (RBF) — і за F1-score, і за balanced accuracy. Імовірна причина в тому, що Tomek прибирає пограничні приклади та зменшує перекриття між класами, тобто робить навчальну вибірку чистішою, а нелінійна модель краще використовує це для побудови гнучкої межі класифікації.


